In [8]:
pip install --user torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.5.1%2Bcu118-cp312-cp312-win_amd64.whl (4.0 MB)
  Using cached https://download.pytorch.org/whl/cu118/torch-2.5.1%2Bcu118-cp312-cp312-win_amd64.whl (2700.1 MB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
##installs pytorch on a cuda-capable windows machine using pip

!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

!pip install torchsummary

!pip install numpy

!pip install matplotlib

!pip install opencv-python

!pip install scikit-learn

!pip install pandas

!pip install pickle5

SyntaxError: invalid syntax (3373289064.py, line 3)

In [10]:
import torch
import torch.nn as nn
import os
import cv2
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')

In [11]:
from torchvision import models
from torchvision import transforms
import os


In [12]:
if(os.path.split(os.getcwd())[1] == "Transfer_Learning" or os.path.split(os.getcwd())[1] == "semi-supervised" or os.path.split(os.getcwd())[1] == "self-supervised"):
    os.chdir("..")
print("Current Working Directory: {}".format(os.path.split(os.getcwd())[1]))

Current Working Directory: progettoVIPM


In [14]:
cuda=False

In [5]:
from torch.utils.data import Dataset
from enum import Enum

class datasets(Enum):
    TRAINING_LABELED = ["train_small.csv", "train_set"]
    TRAINING_UNLABELED = ["train_unlabeled.csv", "train_set"]
    TEST = ["val_info.csv", "val_set"]
    TEST_DEGRADED = ["val_info.csv", "val_set_degraded"]

class networks(Enum):
    ALEXNET = [227, models.alexnet(pretrained=True), "AlexNet"] #227?
    RESNET50 = [224, models.resnet50(pretrained=True), "ResNet50"]
    GOOGLENET = [224, models.googlenet(pretrained=True), "GoogLeNet"]
    MOBILENET = [224, models.mobilenet_v3_small(pretrained=True), "mobilenet_v3_small"]

class ImageDataset(Dataset):
    def __init__(self, dataset, network_input_size):
        super().__init__()
        self.images_names = []
        self.labels = []
        dataset = dataset.value
        annotations_file = dataset[0]
        img_dir = dataset[1]
        with open(annotations_file, newline='') as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                self.images_names.append("./{}/{}".format(img_dir, row[0]))
                self.labels.append(row[1])
        self.images_names = np.array(self.images_names)
        self.labels = np.array(self.labels)
        # in base al valore passato si sceglie la rete che utilizzerà il dataset, serve per modificare le dimensioni delle immagini
        self.im_size = network_input_size
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index):
        #moveaxis serve per avere come dimensione dell'immagine (3, righe, colonne) invece di (righe, colonne, 3)
        image = np.moveaxis(cv2.resize((cv2.imread(self.images_names[index], cv2.IMREAD_COLOR).astype(np.double)/255), 
                                       (self.im_size,self.im_size), 
                                        interpolation=cv2.INTER_CUBIC).astype(np.float32),
                            -1, 0)
        # eventualmente si può aggiungere l'alternativa di fare random cropping dell'immagine
        label = self.labels[index]                
        if(cuda):
            return torch.from_numpy(image).cuda(), label
        else:
            return image, label

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
import numpy as np
from PIL import Image
from torch.utils.data import DataLoader

dataset_holder = ImageDataset(dataset=datasets.TRAINING_UNLABELED, network_input_size=256)
loader = DataLoader(dataset=dataset_holder, shuffle=False, batch_size=1)

imagesToRotate=2000
i=0
rotated_images=[]
rotated_images_y=[]
orientations = [0, 90, 180, 270]
for image, y in tqdm(loader):
    i=i+1
    if i>imagesToRotate: break
    for angle in orientations:
        rotated_image = transforms.functional.rotate(image, angle)
        rotated_images.append(rotated_image)
        rotated_images_y.append(int(angle/90))



  2%|▏         | 2000/113455 [00:35<33:20, 55.71it/s] 


In [16]:
# Convert labels to one-hot encoding
def one_hot_encode(labels, num_classes):
    one_hot_labels = np.zeros((labels.size, num_classes))
    one_hot_labels[np.arange(labels.size), labels] = 1
    return one_hot_labels

rotated_images_y = one_hot_encode(np.array(rotated_images_y), 4)

NameError: name 'rotated_images_y' is not defined

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.utils.data import DataLoader, TensorDataset

# Load the pretrained mobilenet_v3_small model
model = models.mobilenet_v3_small(pretrained=True)

# Modify the classifier to fit 4 classes
model.classifier[3] = nn.Linear(model.classifier[3].in_features, 4)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Prepare the dataset and dataloader
# Assuming rotated_images is a list of tensors with shape [3, 256, 256] (RGB images) WRONG ASSUMPTION WRONG ASSUMPTION WRONG ASSUMPTION WRONG ASSUMPTION
rotated_images_tensor = torch.stack(rotated_images)  # Stack tensors to shape [batch_size, 3, 256, 256]
rotated_images_y_tensor = torch.tensor(rotated_images_y, dtype=torch.long)

dataset = TensorDataset(rotated_images_tensor, rotated_images_y_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)  # Adjust batch size as needed

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(dataloader):
        optimizer.zero_grad()
        outputs = model(torch.squeeze(inputs))
        loss = criterion(outputs.to(torch.float ), labels.to(torch.float ))
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')


[Epoch 1, Batch 100] loss: 0.784
[Epoch 1, Batch 200] loss: 0.476
[Epoch 2, Batch 100] loss: 0.330
[Epoch 2, Batch 200] loss: 0.328
[Epoch 3, Batch 100] loss: 0.229
[Epoch 3, Batch 200] loss: 0.254
[Epoch 4, Batch 100] loss: 0.158
[Epoch 4, Batch 200] loss: 0.193
[Epoch 5, Batch 100] loss: 0.121
[Epoch 5, Batch 200] loss: 0.150
Finished Training


In [8]:
from torch.utils.data import Dataset
from enum import Enum

class datasets(Enum):
    TRAINING_LABELED = ["train_small.csv", "train_set"]
    TRAINING_UNLABELED = ["train_unlabeled.csv", "train_set"]
    TEST = ["val_info.csv", "val_set"]
    TEST_DEGRADED = ["val_info.csv", "val_set_degraded"]

class ImageDataset(Dataset):
    def __init__(self, dataset, network_input_size):
        super().__init__()
        self.images_names = []
        self.labels = []
        dataset = dataset.value
        annotations_file = dataset[0]
        img_dir = dataset[1]
        with open(annotations_file, newline='') as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                self.images_names.append("./{}/{}".format(img_dir, row[0]))
                self.labels.append(row[1])
        self.images_names = np.array(self.images_names)
        self.labels = np.array(self.labels)
        # in base al valore passato si sceglie la rete che utilizzerà il dataset, serve per modificare le dimensioni delle immagini
        self.im_size = network_input_size
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index):
        #moveaxis serve per avere come dimensione dell'immagine (3, righe, colonne) invece di (righe, colonne, 3)
        image = np.moveaxis(cv2.resize((cv2.imread(self.images_names[index], cv2.IMREAD_COLOR).astype(np.double)/255), 
                                       (self.im_size,self.im_size), 
                                        interpolation=cv2.INTER_CUBIC).astype(np.float32),
                            -1, 0)
        # eventualmente si può aggiungere l'alternativa di fare random cropping dell'immagine
        label = self.labels[index]                
        if(cuda):
            return torch.from_numpy(image).cuda(), label
        else:
            return image, label

In [11]:
from copy import deepcopy
model2 = deepcopy(model)

In [12]:
import pickle

# Open a file and use dump() 
with open('modelTrainedOnRotated.pkl', 'wb') as file:       
    # A new file will be created 
    pickle.dump(model, file) 

In [4]:
import pickle

with open('modelTrainedOnRotated.pkl', 'rb') as file: 
      
    # Call load method to deserialze 
    model2 = pickle.load(file) 

In [17]:
dataset_holder = ImageDataset(dataset=datasets.TRAINING_LABELED, network_input_size=256)
loader = DataLoader(dataset=dataset_holder, shuffle=False, batch_size=1)
images =[]
images_y=[]
numOfImagesToFinetune=2000
i=0
for image, y in tqdm(loader):
    i=i+1
    if i>numOfImagesToFinetune: break    
    images.append(image)
    images_y.append(int(y[0]))


images_y = one_hot_encode(np.array(images_y), 251)

images_tensor = torch.stack(images)  # Stack tensors to shape [batch_size, 3, 256, 256]
images_y_tensor = torch.tensor(images_y, dtype=torch.long)

dataset = TensorDataset(images_tensor, images_y_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)  # Adjust batch size as needed


 40%|███▉      | 2000/5020 [00:10<00:16, 184.56it/s]


In [18]:
# Modify the model2 to have 251 output classes
model2.classifier = nn.Linear(576, 251)

# Define the new loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model2.parameters(), lr=0.001)

# Fine-tuning loop
num_epochs = 5  # Adjust the number of epochs as needed
model2.train()

num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(dataloader):
        optimizer.zero_grad()
        outputs = model2(torch.squeeze(inputs))
        loss = criterion(outputs.to(torch.float ), labels.to(torch.float ))
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0



In [59]:
test_dataset_holder = ImageDataset(dataset=datasets.TEST, network_input_size=256)
test_loader = DataLoader(dataset=test_dataset_holder, shuffle=False, batch_size=1)
test_images =[]
test_images_y=[]
numOfImagesToTestWith=3000
i=0
for image, y in tqdm(loader):
    i=i+1
    if i>numOfImagesToTestWith: break    
    test_images.append(image)
    test_images_y.append(int(y[0]))


test_images_y = one_hot_encode(np.array(test_images_y), 251)

test_images_tensor = torch.squeeze(torch.stack(test_images))  # Stack tensors to shape [batch_size, 3, 256, 256]
test_images_y_tensor = torch.tensor(test_images_y, dtype=torch.long)

test_dataset = TensorDataset(test_images_tensor, test_images_y_tensor)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)  # Adjust batch size as needed


 60%|█████▉    | 3000/5020 [00:40<00:27, 73.45it/s]


In [60]:
from sklearn.metrics import accuracy_score
guessedTestLabels = model2(test_images_tensor)

: 

In [ ]:
decodedProba=[]
for e in guessedTestLabels:
    decodedProba.append(np.argmax(e.detach().numpy()))

decodedTestImagesY=[]
for e in guessedTestLabels:
    decodedTestImagesY.append(np.argmax(e.detach().numpy()))


score = accuracy_score(decodedTestImagesY, decodedProba)
print(score)

1.0
